# Election Result Data for US on county level

---

#### Data Source

2016 and 2020 data extracted from https://github.com/tonmcg/US_County_Level_Election_Results_08-20

### Final csv election_2020_results.csv

- shape (3152, 9) - includes 3112 counties and 40 districts of Alaska 
- 2 missing territories - Guam & Puerto Rico do not vote
- use 2020 data as more recent & up to date fips code

Columns of interest:
- fips 
- state 
- county
- votes_dem
- votes_gop
- total_votes
- per_dem
- per_gop
- per_point_diff


In [2]:
import pandas as pd
import requests
import io
import matplotlib.pyplot     as plt
import seaborn               as sns
import numpy as np


In [3]:
url = "https://raw.githubusercontent.com/ralidimitrova/DS4A_team27/master/data/raw/Presidential_Results_2020_US_County_Level_.csv" # Make sure the url is the raw version of the file on GitHub

df20 = pd.read_csv(url, delimiter=',')

df20.shape


(3152, 10)

In [4]:
url = "https://raw.githubusercontent.com/ralidimitrova/DS4A_team27/master/data/raw/Presidential_Results_2016_US_County_Level.csv" # Make sure the url is the raw version of the file on GitHub

df16 = pd.read_csv(url, delimiter=',')

df16.shape

(3141, 11)

##### Checking for missing data

In [5]:
df20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3152 entries, 0 to 3151
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state_name      3152 non-null   object 
 1   county_fips     3152 non-null   int64  
 2   county_name     3152 non-null   object 
 3   votes_gop       3152 non-null   int64  
 4   votes_dem       3152 non-null   int64  
 5   total_votes     3152 non-null   int64  
 6   diff            3152 non-null   int64  
 7   per_gop         3152 non-null   float64
 8   per_dem         3152 non-null   float64
 9   per_point_diff  3152 non-null   float64
dtypes: float64(3), int64(5), object(2)
memory usage: 246.4+ KB


In [6]:
df16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      3141 non-null   int64  
 1   votes_dem       3141 non-null   int64  
 2   votes_gop       3141 non-null   int64  
 3   total_votes     3141 non-null   int64  
 4   per_dem         3141 non-null   float64
 5   per_gop         3141 non-null   float64
 6   diff            3141 non-null   object 
 7   per_point_diff  3141 non-null   object 
 8   state_abbr      3141 non-null   object 
 9   county_name     3141 non-null   object 
 10  combined_fips   3141 non-null   int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 270.1+ KB


No missing values in 2020 and 2016 datasets.

In [7]:
fips2016 = pd.Index(df16.combined_fips)
fips2020 = pd.Index(df20.county_fips)

fips2016.difference(fips2020).values
#list of fips that are in 2016 but not 2020



array([ 2013,  2016,  2020,  2050,  2060,  2068,  2070,  2090,  2100,
        2105,  2110,  2122,  2130,  2150,  2164,  2170,  2180,  2185,
        2188,  2195,  2198,  2220,  2230,  2240,  2261,  2270,  2275,
        2282,  2290, 46113])

In [8]:
fips2016 = pd.Index(df16.combined_fips)
fips2020 = pd.Index(df20.county_fips)

fips2020.difference(fips2016).values
#list of fips that are in 2020 but not 2016

array([ 2901,  2902,  2903,  2904,  2905,  2906,  2907,  2908,  2909,
        2910,  2911,  2912,  2913,  2914,  2915,  2916,  2917,  2918,
        2919,  2920,  2921,  2922,  2923,  2924,  2925,  2926,  2927,
        2928,  2929,  2930,  2931,  2932,  2933,  2934,  2935,  2936,
        2937,  2938,  2939,  2940, 46102])

Oglala Lakota county was renamed and had its fips code changed from 2016 to 2020. It changed from 46113 to 46102. Alaska split into different areas/ fips codes for 2016 to 2020 data.

In [9]:
url = 'https://raw.githubusercontent.com/ralidimitrova/DS4A_team27/master/data/clean/vaccination_rate_US.csv'
df3 = pd.read_csv(url, delimiter=',')
df3.head()

,fips,state,county,state_abb,popfull_per,pop_full_12plus_per,pop_full_18plus_per,pop_full_65plus_per,pop_first_per,pop_first_12plus_per,pop_first_18plus_per,pop_first_65plus_per
0,1001,Alabama,Autauga County,AL,34.0,39.9,41.8,61.7,43.1,50.6,52.6,74.5
1,1003,Alabama,Baldwin County,AL,42.7,49.5,51.9,75.3,54.4,63.0,65.7,91.1
2,1005,Alabama,Barbour County,AL,35.7,41.1,43.0,66.1,46.6,53.8,56.0,76.3
3,1007,Alabama,Bibb County,AL,29.7,34.2,35.7,57.0,37.3,42.8,44.7,64.5
4,1009,Alabama,Blount County,AL,26.4,31.0,32.9,49.5,32.8,38.5,40.6,57.3


In [10]:
fips2016 = pd.Index(df16.combined_fips)
fipsvaccine = pd.Index(df3.fips)

fipsvaccine.difference(fips2016).values
#list of fips that are in vaccine but not 2016

array([ 2158, 46102, 66010, 72001, 72003, 72005, 72007, 72009, 72011,
       72013, 72015, 72017, 72019, 72021, 72023, 72025, 72027, 72029,
       72031, 72033, 72035, 72037, 72039, 72041, 72043, 72045, 72047,
       72049, 72051, 72053, 72054, 72055, 72057, 72059, 72061, 72063,
       72065, 72067, 72069, 72071, 72073, 72075, 72077, 72079, 72081,
       72083, 72085, 72087, 72089, 72091, 72093, 72095, 72097, 72099,
       72101, 72103, 72105, 72107, 72109, 72111, 72113, 72115, 72117,
       72119, 72121, 72123, 72125, 72127, 72129, 72131, 72133, 72135,
       72137, 72139, 72141, 72143, 72145, 72147, 72149, 72151, 72153])

Missing counties:2158 (Alaska), 46102 (Renamed county), 66010 (Guam - do not vote), 72001 - 72153 (Puerto Rico - do not vote)
    

In [11]:
fips2020 = pd.Index(df20.county_fips)
fipsvaccine = pd.Index(df3.fips)

fipsvaccine.difference(fips2020).values
#list of fips that are in vaccine but not 2020

array([ 2013,  2016,  2020,  2050,  2060,  2068,  2070,  2090,  2100,
        2105,  2110,  2122,  2130,  2150,  2158,  2164,  2170,  2180,
        2185,  2188,  2195,  2198,  2220,  2230,  2240,  2261,  2275,
        2282,  2290, 66010, 72001, 72003, 72005, 72007, 72009, 72011,
       72013, 72015, 72017, 72019, 72021, 72023, 72025, 72027, 72029,
       72031, 72033, 72035, 72037, 72039, 72041, 72043, 72045, 72047,
       72049, 72051, 72053, 72054, 72055, 72057, 72059, 72061, 72063,
       72065, 72067, 72069, 72071, 72073, 72075, 72077, 72079, 72081,
       72083, 72085, 72087, 72089, 72091, 72093, 72095, 72097, 72099,
       72101, 72103, 72105, 72107, 72109, 72111, 72113, 72115, 72117,
       72119, 72121, 72123, 72125, 72127, 72129, 72131, 72133, 72135,
       72137, 72139, 72141, 72143, 72145, 72147, 72149, 72151, 72153])

Missing counties: 2013-2290 (Alaska - different fips codes in 2020 election dataset), 66010 (Guam - do not vote) ,72001-72153 (Puerto Rico - do not vote)

##### Clean Data

In [12]:
del df16['Unnamed: 0']

df16

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abbr,county_name,combined_fips
0,93003,130413,246588,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2013
1,93003,130413,246588,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2016
2,93003,130413,246588,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2020
3,93003,130413,246588,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2050
4,93003,130413,246588,0.377159,0.528870,"37,410",15.17%,AK,Alaska,2060
...,...,...,...,...,...,...,...,...,...,...
3136,3233,12153,16661,0.194046,0.729428,"8,920",53.54%,WY,Sweetwater County,56037
3137,7313,3920,12176,0.600608,0.321945,"3,393",27.87%,WY,Teton County,56039
3138,1202,6154,8053,0.149261,0.764187,"4,952",61.49%,WY,Uinta County,56041
3139,532,2911,3715,0.143203,0.783580,"2,379",64.04%,WY,Washakie County,56043


In [13]:
mapping2 = {'combined_fips': 'fips'}
df16.rename(columns = mapping2, inplace = True)

mapping3 = {'state_abbr': 'state_abb'}
df16.rename(columns = mapping3, inplace = True)

mapping4 = {'county_name': 'county'}
df16.rename(columns = mapping4, inplace = True)

df16.head()

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abb,county,fips
0,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2013
1,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2016
2,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2020
3,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2050
4,93003,130413,246588,0.377159,0.52887,"37,410",15.17%,AK,Alaska,2060


In [14]:
df16 = df16.sort_values(by=['fips'])
df16.head()

,votes_dem,votes_gop,total_votes,per_dem,per_gop,diff,per_point_diff,state_abb,county,fips
29,5908,18110,24661,0.239569,0.734358,"12,202",49.48%,AL,Autauga County,1001
30,18409,72780,94090,0.195653,0.773515,"54,371",57.79%,AL,Baldwin County,1003
31,4848,5431,10390,0.466603,0.522714,583,5.61%,AL,Barbour County,1005
32,1874,6733,8748,0.214220,0.769662,"4,859",55.54%,AL,Bibb County,1007
33,2150,22808,25384,0.084699,0.898519,"20,658",81.38%,AL,Blount County,1009


In [15]:
df16 = df16[['fips', 'county', 'state_abb', 'votes_dem', 'votes_gop', 'total_votes', 'per_dem', 'per_gop', 'per_point_diff']]
df16 #relevent columns in same order as vaccine dataset - not sure how to reorder the first column numbers

,fips,county,state_abb,votes_dem,votes_gop,total_votes,per_dem,per_gop,per_point_diff
29,1001,Autauga County,AL,5908,18110,24661,0.239569,0.734358,49.48%
30,1003,Baldwin County,AL,18409,72780,94090,0.195653,0.773515,57.79%
31,1005,Barbour County,AL,4848,5431,10390,0.466603,0.522714,5.61%
32,1007,Bibb County,AL,1874,6733,8748,0.214220,0.769662,55.54%
33,1009,Blount County,AL,2150,22808,25384,0.084699,0.898519,81.38%
...,...,...,...,...,...,...,...,...,...
3136,56037,Sweetwater County,WY,3233,12153,16661,0.194046,0.729428,53.54%
3137,56039,Teton County,WY,7313,3920,12176,0.600608,0.321945,27.87%
3138,56041,Uinta County,WY,1202,6154,8053,0.149261,0.764187,61.49%
3139,56043,Washakie County,WY,532,2911,3715,0.143203,0.783580,64.04%


In [16]:
mapping5 = {'county_fips': 'fips'}
df20.rename(columns = mapping5, inplace = True)

mapping6 = {'state_name': 'state'}
df20.rename(columns = mapping6, inplace = True)

mapping7 = {'county_name': 'county'}
df20.rename(columns = mapping7, inplace = True)

df20.head()

,state,fips,county,votes_gop,votes_dem,total_votes,diff,per_gop,per_dem,per_point_diff
0,Alabama,1001,Autauga County,19838,7503,27770,12335,0.714368,0.270184,0.444184
1,Alabama,1003,Baldwin County,83544,24578,109679,58966,0.761714,0.224090,0.537623
2,Alabama,1005,Barbour County,5622,4816,10518,806,0.534512,0.457882,0.076631
3,Alabama,1007,Bibb County,7525,1986,9595,5539,0.784263,0.206983,0.577280
4,Alabama,1009,Blount County,24711,2640,27588,22071,0.895716,0.095694,0.800022


In [23]:
df20 = df20[['fips', 'state', 'county', 'votes_dem', 'votes_gop', 'total_votes', 'per_dem', 'per_gop','per_point_diff']]
df20
df = df20
df#relevent columns in same order as vaccine dataset - not sure how to reorder the first column numbers

,fips,state,county,votes_dem,votes_gop,total_votes,per_dem,per_gop,per_point_diff
0,1001,Alabama,Autauga County,7503,19838,27770,0.270184,0.714368,0.444184
1,1003,Alabama,Baldwin County,24578,83544,109679,0.224090,0.761714,0.537623
2,1005,Alabama,Barbour County,4816,5622,10518,0.457882,0.534512,0.076631
3,1007,Alabama,Bibb County,1986,7525,9595,0.206983,0.784263,0.577280
4,1009,Alabama,Blount County,2640,24711,27588,0.095694,0.895716,0.800022
...,...,...,...,...,...,...,...,...,...
3147,56037,Wyoming,Sweetwater County,3823,12229,16603,0.230260,0.736554,0.506294
3148,56039,Wyoming,Teton County,9848,4341,14677,0.670982,0.295769,-0.375213
3149,56041,Wyoming,Uinta County,1591,7496,9402,0.169219,0.797277,0.628058
3150,56043,Wyoming,Washakie County,651,3245,4012,0.162263,0.808824,0.646560


#### Save data

In [26]:
df.to_csv('./election_results_us.csv', index = False)